In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from IPython import display
from tensorflow.python.client import device_lib
import os
import re

In [2]:
print(device_lib.list_local_devices())
gpus = tf.config.list_physical_devices('GPU')
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11183533259735920895
xla_global_id: -1
]


In [3]:
def return_DataPathList(path="annotation/anno.csv"):
    f = open(path, mode="r", encoding="UTF-8")
    tmp = f.readlines()
    datalist = []
    for dpath in tmp:
        datalist.append(dpath.split(",")[0])
    f.close()
    return datalist

In [4]:
def return_Datalabel(path="annotation/anno.csv"):
    f = open(path, mode="r", encoding="UTF-8")
    tmp = f.readlines()
    datalist = []
    
    for dpath in tmp:
        tmp2 = dpath.split(",")[1].replace("\n", "")
        datalist.append([int(i) for i in tmp2])
    return datalist

In [28]:
# 学習データのパス
datalist_train = return_DataPathList(path="annotation/35anno_train_devidecomplate.csv")
# 学習データのラベル
train_label = return_Datalabel(path="annotation/35anno_train_devidecomplate.csv")
# テストデータのパス
datalist_test = return_DataPathList(path="annotation/35anno_test_devidecomplate.csv")
# テストデータのラベル
test_label = return_Datalabel(path="annotation/35anno_test_devidecomplate.csv")

train_data = [] # 学習データを格納する
test_data = [] #　テストデータを格納する
tmp_data = 0 # 一時的に座標データを格納する
max = 0
for i in range(len(datalist_train)):
    tmp_data = pd.read_csv(datalist_train[i], header=None).values

    train_data.append(tmp_data)
for i in range(len(datalist_test)):
    tmp_data = pd.read_csv(datalist_test[i], header=None).values

    test_data.append(tmp_data)

In [29]:
train_data = np.array(train_data).astype(np.float32)
test_data = np.array(test_data).astype(np.float32)
train_label = np.array(train_label).astype(np.int64)
test_label = np.array(test_label).astype(np.int64)
train_data = np.expand_dims(train_data, -1)
test_data = np.expand_dims(test_data, -1)
test_label = keras.utils.to_categorical(test_label, 2)
train_label = keras.utils.to_categorical(train_label, 2)

In [30]:
dim1 = np.array([[1]]*len(train_data)).astype(np.int64)
dim2 = np.array([[1]]*len(test_data)).astype(np.int64)
#print(dim)
print(len(train_data[0]), len(train_data[0][0]), len(train_data), len(test_label))

15 14 1535 94


In [31]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_label))
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_label))

In [32]:
# データのシャッフル
BATCH_SIZE = 10
SHUFFLE_BUFFER_SIZE = 1000000
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
#test_dataset = test_dataset.batch(BATCH_SIZE)
print(train_dataset)

<BatchDataset element_spec=(TensorSpec(shape=(None, 15, 14, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>


In [33]:
print(test_dataset)

<TensorSliceDataset element_spec=(TensorSpec(shape=(15, 14, 1), dtype=tf.float32, name=None), TensorSpec(shape=(2,), dtype=tf.float32, name=None))>


In [42]:
def build_model(input_dim, output_dim, rnn_layers=5, rnn_units=128):
    """Model similar to DeepSpeech2."""
    # Model's input
    input_model = layers.Input((input_dim[0], input_dim[1]), name="input")
    # Expand the dimension to use 2D CNN.
    x = layers.Reshape((-1, input_dim[0], 1), name="expand_dim")(input_model)
    
    x = layers.ZeroPadding2D(padding=1, data_format=None)(x)
    x = layers.Conv2D(32, kernel_size=(3, 3), activation="relu")(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    x = layers.ZeroPadding2D(padding=1, data_format=None)(x)
    x = layers.Conv2D(64, kernel_size=(3, 3), activation="relu")(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.25)(x)
    # Classification layer
    output = layers.Dense(units=output_dim, activation="softmax")(x)
    # Model
    model = keras.Model(input_model, output, name="Shogi-winner-model")
    # Optimizer
    opt = keras.optimizers.Adam(learning_rate=1e-6)
    # Compile the model and return
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=["accuracy"])
    return model


model = build_model(
    input_dim=[15,14],
    output_dim=2,
    rnn_units=512
)
model.summary(line_length=110)

Model: "Shogi-winner-model"
______________________________________________________________________________________________________________
 Layer (type)                                    Output Shape                                Param #          
 input (InputLayer)                              [(None, 15, 14)]                            0                
                                                                                                              
 expand_dim (Reshape)                            (None, 14, 15, 1)                           0                
                                                                                                              
 zero_padding2d_4 (ZeroPadding2D)                (None, 16, 17, 1)                           0                
                                                                                                              
 conv2d_4 (Conv2D)                               (None, 14, 15, 32)                 

In [35]:
%load_ext tensorboard
!rd /s /q logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


指定されたファイルが見つかりません。


In [36]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
tensorboard_callback = []

In [43]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_label))
#test_dataset = test_dataset.batch()
#validation_callback = CallbackEval(test_dataset)
model.fit(train_dataset, epochs=100, batch_size=1)

Epoch 1/100
154/154 [==============================] - 4s 12ms/step - loss: 1.3189 - accuracy: 0.5147
Epoch 2/100
154/154 [==============================] - 2s 11ms/step - loss: 1.2297 - accuracy: 0.5336
Epoch 3/100
154/154 [==============================] - 2s 11ms/step - loss: 1.2778 - accuracy: 0.5231
Epoch 4/100
154/154 [==============================] - 2s 11ms/step - loss: 1.2504 - accuracy: 0.5205
Epoch 5/100
154/154 [==============================] - 2s 11ms/step - loss: 1.2486 - accuracy: 0.5049
Epoch 6/100
154/154 [==============================] - 2s 11ms/step - loss: 1.2162 - accuracy: 0.5147
Epoch 7/100
154/154 [==============================] - 2s 11ms/step - loss: 1.2217 - accuracy: 0.5186
Epoch 8/100
154/154 [==============================] - 2s 11ms/step - loss: 1.2564 - accuracy: 0.4971
Epoch 9/100
154/154 [==============================] - 2s 11ms/step - loss: 1.2423 - accuracy: 0.5199
Epoch 10/100
154/154 [==============================] - 2s 10ms/step - loss: 1.221

In [60]:
predictions = []
mydata = []
targets = []
piyo = []
tmpl = []
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_label))
test_dataset = test_dataset.batch(1)
count = 0

for batch in test_dataset:
    X, y = batch
    batch_predictions = model.predict(X)
    predictions.extend(batch_predictions)

    for label in y:
        targets.append(label)

    mydata.extend(X[0].numpy()[13][0])
    tmpl.append(X[0][12][2])
    piyo.append(X[0])
    #if count == 100:
     #   break
    for i in range(len(predictions)):
        print(f"Target       : {targets[i]}")
        print(f"machineScore : {mydata[i]}")
        print(f"machineScore : {mydata[i]*5000}")
        print(f"Prediction   : {predictions[i]}")
        
        print("-" * 100)
    
    count = count+1


1/1 [==============================] - 0s 22ms/step
Target       : [1. 0.]
machineScore : -0.05700000002980232
machineScore : -285.0000001490116
Prediction   : [0.6893842  0.31061575]
----------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 21ms/step
Target       : [1. 0.]
machineScore : -0.05700000002980232
machineScore : -285.0000001490116
Prediction   : [0.6893842  0.31061575]
----------------------------------------------------------------------------------------------------
Target       : [1. 0.]
machineScore : 0.00019999999494757503
machineScore : 0.9999999747378752
Prediction   : [0.6832879  0.31671208]
----------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 17ms/step
Target       : [1. 0.]
machineScore : -0.05700000002980232
machineScore : -285.0000001490116
Prediction   : [0.6893842  0.31061575]
-----------

In [39]:
class CallbackEval(keras.callbacks.Callback):
    """Displays a batch of outputs after every epoch."""

    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset

    def on_epoch_end(self, epoch: int, logs=None):
        predictions = []
        targets = []
        for batch in self.dataset:
            X, y = batch
            batch_predictions = model.predict(X)
            predictions.extend(batch_predictions)
            
            for label in y:
                targets.append(label)
        
            for i in range(len(predictions)):
                print(f"Target       : {targets[i]}")
                print(f"machineScore : {mydata[i]}")
                print(f"Prediction   : {predictions[i]}")

                print("-" * 100)


In [45]:
#mydata = []
len(test_dataset), len(mydata)
for i in range(len(piyo)):
    if abs(piyo[i][13][0][0].numpy()*5000) > 1000:
        print(i+1)
        print(predictions[i])
        print(piyo[i][13][0].numpy()*5000)
        print("-"*30)

9
[0.8170415  0.18295841]
[1289.9999]
------------------------------
10
[0.622929 0.377071]
[1191.]
------------------------------
11
[0.68808496 0.31191504]
[3022.]
------------------------------
15
[0.92565817 0.07434181]
[1318.]
------------------------------
19
[0.752577   0.24742301]
[-1740.]
------------------------------
21
[0.65148824 0.34851176]
[1073.]
------------------------------
27
[0.6694996  0.33050045]
[-2464.]
------------------------------
38
[0.7361205  0.26387948]
[4173.]
------------------------------
39
[0.88334334 0.11665668]
[-1310.]
------------------------------
40
[0.24579935 0.75420064]
[-1270.]
------------------------------
48
[0.7383155  0.26168454]
[1213.]
------------------------------
49
[0.82688296 0.17311701]
[3425.]
------------------------------


In [41]:
koma_dic = {"K":1, "R":2, "B":3, "G":4, "S":5, "N": 6, "L": 7, "P": 8,
           "k": -1, "r": -2, "b":-3, "g":-4, "s": -5, "n": -6, "l": -7, "p": -8}
mochi_id = {"R":0, "B": 1, "G": 2, "S":3, "N": 4, "L": 5, "P": 6,
           "r":7, "b": 8, "g":9, "s":10, "n":11, "l":12, "p":13}
nari_dic = {"R":9, "B":10, "r": -9, "b":-10,
            "S":4, "N":4, "L":4, "P":4,
            "s":-4, "n":-4, "l":-4, "p":-4}
nari_dic_inv = {9:"竜", 10:"馬", -9:"v竜", -10:"v馬"}

# normalize
koma_dic_n = {"K":1/10, "R":2/10, "B":3/10, "G":4/10, "S":5/10, "N": 6/10, "L": 7/10, "P": 8/10,
           "k": -1/10, "r": -2/10, "b":-3/10, "g":-4/10, "s": -5/10, "n": -6/10, "l": -7/10, "p": -8/10}

nari_dic_n = {"R":9/10, "B":10/10, "r": -9/10, "b":-10/10,
            "S":4/10, "N":4/10, "L":4/10, "P":4/10,
            "s":-4/10, "n":-4/10, "l":-4/10, "p":-4/10}

In [20]:
koma_dic_inv = {v:k for k, v in koma_dic_n.items()}
nari_dic_inv = {v:k for k, v in nari_dic_n.items()}
decode_koma = {"K":" 玉", "R":" 飛", "B":" 角", "G":" 金", "S":" 銀", "N": " 桂", "L": " 香", "P": " 歩",
           "k": "v王", "r": "v飛", "b":"v角", "g":"v金", "s": "v銀", "n": "v桂", "l": "v香", "p": "v歩"}
number_dic = {1:"一", 2:"二", 3:"三", 4:"四", 5:"五", 6:"六", 7:"七", 8:"八", 9:"九", 10:"十"}

In [21]:
def shogi_table(vec):
    print("後手の持駒：", end="")
    for i in range(7,14):
        #print(vec.numpy()[10][i][0])
        if vec.numpy()[10][i][0]==1:
            print(f"{decode_koma[mochi_id_inv[i*10]][1]}", end=" ")
        #elif int(vec.numpy()[10][i][0])!=0:
            #print(f"{decode_koma[mochi_id_inv[i]][1]}{number_dic[int(vec.numpy()[10][i][0])]}", end=" ")
    print("\n  ９ ８ ７ ６ ５ ４ ３ ２ １")
    print("+---------------------------+")
    for j in range(9):
        print("|", end="")
        try:
            for k in vec.numpy()[j][:9]:
                if 0< abs(k.item()) < 9:
                    koma = koma_dic_inv[k.item()]
                    print(decode_koma[koma], end="")
                elif abs(k.item()) > 8:
                    print(nari_dic_inv[k.item()], end="")
                else:
                    print(" ・", end="")
            #print("/", end="")
            print(f"|{number_dic[j+1]}")
        except:
            print(f"|{number_dic[j+1]}")
    print("+---------------------------+")
    print("先手の持駒：", end="")
    for i in range(6):
        #print(vec.numpy()[9][i][0])
        if vec.numpy()[10][i][0]==1:
            print(f"{decode_koma[mochi_id_inv[i]]}", end=" ")
        #elif vec.numpy()[10][i][0] != 0:
         #   print(f"{decode_koma[mochi_id_inv[i]]}{number_dic[int(vec.numpy()[10][i][0])]}", end=" ")
    print()
for i in piyo[13].numpy()[0]:
    print(i[0])

-0.7
-0.6
-0.5
-0.4
-0.1
-0.4
0.0
-0.6
-0.7
0.0
0.0
0.0
0.0
0.0


In [22]:
#for i in piyo[101].numpy()[0]:
   # for j in i:
        #print(j, end="")
    #print()
#print(piyo[13].numpy())
print(piyo[13].numpy()[0][13][0])

0.0


In [23]:
mydata[0]*5000

32.999999821186066

In [24]:
for i in range(len(predictions)):
    #print(piyo[i][0][0])
    #if i == 100:
        #print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n"*50)
    #if mydata[i]*5000 > 1000:
    #shogi_table(piyo[i][0])
    print(f"評価値       :{piyo[i].numpy()[0][13][0][0]}")
    print(f"評価値       :{piyo[i].numpy()[0][13][0][0]*5000}")
    print(f"Target       : {targets[i]}")
    print(f"machineScore : {mydata[i]}")
    print(f"Prediction   : {predictions[i]}")
    print("-" * 100)
#for i in piyo:
    #print(piyo[i].numpy()[0][12][2]*5000)

IndexError: invalid index to scalar variable.

In [61]:
data_num = len(targets)
origin_acc = 0
model_acc = 0
Blackwin = 0
for i in range(len(targets)):
    tmp = 1 if predictions[i][0] >= 0.5 else 0
    if mydata[i] >= 0:
        flag = 0
    else:
        flag = 1
    if flag == targets[i][0]:
        origin_acc = origin_acc+1
    if tmp == targets[i][0]:
        model_acc = model_acc + 1
    if targets[i][0] == 0:
        Blackwin = Blackwin+1
print(f"評価値のみの勝敗予測            :  {origin_acc/data_num * 100}%")
print(f"対局条件を加えたモデルの勝敗予測 :  {model_acc/data_num * 100}%")
print(f"常に先手が勝ちと予測した場合の勝敗予測 :  {Blackwin/data_num * 100}%")

評価値のみの勝敗予測            :  23.404255319148938%
対局条件を加えたモデルの勝敗予測 :  63.829787234042556%
常に先手が勝ちと予測した場合の勝敗予測 :  37.234042553191486%


In [25]:
test_loss = model.evaluate(test_dataset)

169/169 [==============================] - 1s 2ms/step - loss: 0.7331 - accuracy: 0.5148


In [ ]:
len(piyo)

In [71]:
model_dir = "shogi-predict-model/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model.save(model_dir+".h5")